# Fitnotes Processing Notebook
This notebook processes workout data manually collected from the Fitnotes workout Android application, creating four tables:
- Sets, showing set-level data during the workout.
- Volume, showing aggregate set, rep, and volume metrics for each exercise.
- Workouts, showing aggregate workout metadata.

In [139]:
#basic imports
import numpy as np
import pandas as pd

from glob import glob
from sqlalchemy import create_engine
engine = create_engine('sqlite:///../gains.db')

In [128]:
def format_columns(df):
    '''
    Set columns to lowercase, strip, and replace spaces with underscores
    '''
    
    df.columns = (df.columns
                  .str.strip()
                  .str.lower()
                  .str.replace(' \(lbs\)', '') #remove the (lbs) subtitle from weight column
                  .str.replace(' ', '_') 
                 )
    return df

def Workouts(df):
    '''
    Create matrix of aggregated key performance indicators for workouts
    '''

    # get key workout performance indicators
    agg = {
        'volume': 'sum', 
        'set_completed': 'mean',
        'id': 'count',
    }

    gains = (df
             .groupby('date', as_index = False)
             .agg(agg)
             .rename(columns = {'set_completed': 'completion_rate',  # rename set_completed to completion_rate because all sets comprise the workout
                                'id': 'sets'
                               })
            )
    
    gains['weekday'] = gains.date.dt.weekday_name
    gains['primary_lift'] = df.groupby('date', as_index = False).apply(lambda g: g.set_index('exercise').weight.idxmax()).fillna('Cardio')
    gains['category'] = df.groupby('date', as_index = False).apply(lambda g: g.set_index('category').volume.idxmax()).fillna('Cardio')
    gains['days_since_workout'] = gains.date.diff().fillna(pd.Timedelta('0 days')).dt.days
    gains['days_since_category'] = gains.groupby('category').date.diff().fillna(pd.Timedelta('0 days')).dt.days
    
    gains = pd.merge_ordered(gains, df[['date', 'category', 'routine']], 
                             on = ['date', 'category'],
                             how = 'left',
                            ).drop_duplicates()
    
    gains = gains[['date', 'weekday', 'routine', 'category', 'primary_lift', 'sets', 'volume', 'completion_rate', 'days_since_workout', 'days_since_category']]
    
    return gains

def Volume(df):
    '''
    Aggregate statistics for exercises within a workout
    '''

    agg = {'category': max,
           'routine': max,
           'reps': sum, 
           'volume': sum, 
           'relative_volume': sum,
           'intensity': 'mean',
           'id': 'count'}
    
    volume = (df
                 .groupby(['date', 'exercise'], as_index = False)[list(agg.keys())]
                 .agg(agg)
                 .rename(columns = {'id': 'sets', 'intensity': 'avg_intensity'})
                 .query('volume > 0'))
    
    # convert reps to int
    volume.reps = volume.reps.astype(int)
    
    volume = volume[['date', 'routine', 'category', 'exercise', 'sets', 'reps', 'avg_intensity', 'volume', 'relative_volume']]
    
    return volume

In [149]:
def assign_routines(df, routines):
    '''
    Assign workout routine to each workout
    '''
    # for every dimension of the routines table...
    cols = ['routine'] # , 'superset', 'sub_superset']

    # ...instantiate dimension to nan, then...
    for col in cols:
        df[col] = np.nan

        # for every exercise in a routine... (each row is an exercise)
        for idx, row in routines.iterrows():

            # if a performed exercise occurs within the dates of that routine's exercise
            exercise_within_dates = (df.exercise == row.exercise) & df.date.between(row.start_date, row.end_date)

            # ...assign the corresponding column
            df[col] = df[col].mask(exercise_within_dates, row[col])

    return df

def add_bodyweight_exercises(df, weight):
    '''
    Add bodyweight to exercises where I lift my whole body
    '''

    # merge dataframe with bodyweight within 60 days of measurement
    df = pd.merge_asof(df, weight[['date', 'bodyweight']], on = 'date', direction = 'backward', tolerance = pd.Timedelta('60 days'))

    # add my bodyweight to the weight lifted on bodyweight exercises
    bodyweight_exercises = ['Pull Up', 'Chin Up', 'Close Grip Dip']
    is_bodyweight = df.exercise.isin(bodyweight_exercises)
    df.loc[is_bodyweight, 'weight'] += df.loc[is_bodyweight, 'bodyweight']

    # drop the bodyweight column
    df = df.drop(['bodyweight'], axis = 1)

    return df

def calculate_one_rep_max(df):
    '''
    Calculate the largest one rep max in the past 30 days.

    This calculation is distinctly separate from calculating 
    the largest one rep max in the past 30 workouts. Using 30 days
    instead of 30 workouts helps account for my strength gains
    or decreases within a time span, assuming I continue to
    lift weights with some degree of consistency. The 30 workouts
    metric has a disadvantage in that it does not account for 
    workouts that occured far away from each other in time.
    This could cause me to overestimate my one rep max if
    the last big max was from a workout a long time ago.
    '''

    def helper(group):
        '''
        Helper function to calculate largest one rep max in past 30 days.

        Creating this function allows me to keep one rep max calcualtions
        within the same functional unit.
        '''
        # calculate the heighest one rep max of the day
        reduced = group.groupby('date').one_rep_max.max()

        # create a date range from the beginning to the end of the exercise's lifespan ...
        date_range = pd.date_range(start = reduced.index.min(), end = reduced.index.max(), freq = 'D')

        # ... then reindex using the daily date range to find the largest one rep max in the last 30 days
        rolling_max = (reduced
                       .reindex(date_range)
                       .rolling('30D')
                       .max()
                       .round()
                       .to_frame('one_rep_max'))

        # update one rep max with the new rolling values; all happens inplace
        group.set_index('date', inplace = True)
        group.update(rolling_max)

        return group

    # create a set id to sort on after groupby
    df['id'] = df.index

    # calculate Epley-formulated one rep max
    df['one_rep_max'] = df.eval('weight * (1 + reps / 30)')

    # calculate latest one rep max within 30 days
    result = (df
              .groupby('exercise')
              .apply(helper)
              .drop('exercise', axis = 1)
              .sort_values('id')
              .reset_index())

    return result    

def volume_intensity(df):
    '''
    Calculate intensity, volume, and relative volume.
    '''

    # calculate intensity  
    df['intensity'] = df.eval('weight / one_rep_max')

    # calculate total volume and ...
    df['volume'] = df.eval('weight * reps')

    # ... and relative volume
    df['relative_volume'] = df.eval('weight * reps * intensity')

    return df

def other_features(df):
    '''
    Impute and format remaining features.
    '''

    # format dates
    df.date = pd.to_datetime(df.date)

    # calculate speed for cardio in units per second
    df.time = pd.to_datetime(df.time)
    df.time = df.time.dt.minute * 60 + df.time.dt.second
    # df['speed'] = df.eval('distance / time')

    # determine completed sets for weights and cardio
    df['set_complete'] = 0
    df.loc[(df.category == 'Cardio') & (df.distance > 0), 'set_complete'] = 1
    df.loc[df.weight > 0, 'set_complete'] = 1

    # make null set_completed before I began officially tracking that metric
    df.loc[df.date < '5-18-18', 'set_complete'] = np.nan
    
    # eliminate incomplete sets for now
    df = df.query('set_complete != 0').reset_index(drop = True)
    
    df['id'] = df.index

    ### prepare for export ###

    # set column order
    col_order = ['id', 'date', 'routine', 'category', 'exercise', 
                 'weight', 'reps', 'intensity', 'one_rep_max', 'volume', 'relative_volume',
                 'distance_unit', 'distance', 'time', 'comment']

    df = df[col_order]

    return df

In [3]:
saved_workouts = glob.glob('exports/Fitnotes*.csv')

In [156]:
routines = pd.read_csv('routines.csv', parse_dates = ['start_date', 'end_date'])
fitnotes = pd.read_csv(saved_workouts[-1]).pipe(format_columns)
weight = pd.read_sql('weight', con = engine, parse_dates = 'date').rename(columns = {'weight':'bodyweight'})

In [150]:
sets = (fitnotes
        .pipe(assign_routines, routines)
        .pipe(add_bodyweight_exercises, weight)
        .pipe(calculate_one_rep_max)
        .pipe(volume_intensity)
        .pipe(other_features))

In [157]:
sets.tail()

,id,date,routine,category,exercise,weight,reps,intensity,one_rep_max,volume,relative_volume,distance_unit,distance,time,comment
6579,6579,2019-06-21,Episode VI: Return of the Legs,Chest,Incline Barbell Bench Press,125.0,12.0,0.668449,187.0,1500.0,1002.673797,NaN,NaN,NaN,NaN
6580,6580,2019-06-21,Episode VI: Return of the Legs,Triceps,Close Grip Dip,204.7,12.0,0.713240,287.0,2456.4,1752.003763,NaN,NaN,NaN,NaN
6581,6581,2019-06-21,Episode VI: Return of the Legs,Triceps,Close Grip Dip,204.7,12.0,0.713240,287.0,2456.4,1752.003763,NaN,NaN,NaN,NaN
6582,6582,2019-06-21,Episode VI: Return of the Legs,Biceps,Decline Dumbbell Hammer Curls,22.5,12.0,0.725806,31.0,270.0,195.967742,NaN,NaN,NaN,NaN
6583,6583,2019-06-21,Episode VI: Return of the Legs,Biceps,Decline Dumbbell Hammer Curls,22.5,12.0,0.725806,31.0,270.0,195.967742,NaN,NaN,NaN,NaN


In [152]:
volume = Volume(sets)

In [153]:
volume.head()

,date,routine,category,exercise,sets,reps,avg_intensity,volume,relative_volume
0,2014-06-07,NaN,Back,Lat Pulldown,3,30,0.750000,4500.0,3375.000000
2,2014-06-07,NaN,Back,Seated Back Extension,3,30,0.750000,3600.0,2700.000000
3,2014-06-08,NaN,Chest,Flat Dumbbell Bench Press,3,30,0.753425,1650.0,1243.150685
5,2014-06-10,NaN,Legs,Leg Press,3,30,0.750000,6750.0,5062.500000
6,2014-06-10,NaN,Legs,Seated Leg Curl Machine,3,30,0.732143,6150.0,4508.035714


In [158]:
sets.to_sql('sets', con = engine, if_exists = 'replace', index = False)
volume.to_sql('volume', con = engine, if_exists = 'replace', index = False)
routines.to_sql('routines', con = engine, if_exists = 'replace', index = False)
# workouts = Workouts(sets)
# workouts.to_sql('workouts', con = engine, if_exists = 'replace', index = False)